In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from torch.nn.attention.flex_attention import flex_attention, create_block_mask, BlockMask
import torch

from gpt_fast.mask_utils import make_base_mask

def causal_mask(b, h, q_idx, kv_idx):
    return q_idx >= kv_idx

device = torch.device("cuda")

BLOCK = 4
B, H, PREFILL_LEN, D = 1, 8, 64, 16
start_inds = BLOCK * torch.arange(B, device=device)
GEN_SEQ_LEN = 128
mask = make_base_mask(start_inds, GEN_SEQ_LEN, device=device, compile=False, BLOCK_SIZE=(BLOCK, BLOCK))

prefill_q = torch.randn(B, H, PREFILL_LEN, D, device=device)
gen_kv = torch.randn(B, H, GEN_SEQ_LEN, D, device=device)
gen_q = torch.randn(B, H, 1, D, device=device)

print(mask[:, 0].to_string())

(0,)
░░                              
██░░                            
████░░                          
██████░░                        
████████░░                      
██████████░░                    
████████████░░                  
██████████████░░                
████████████████░░              
██████████████████░░            
████████████████████░░          
██████████████████████░░        
████████████████████████░░      
██████████████████████████░░    
████████████████████████████░░  
██████████████████████████████░░



In [31]:
prefill_mask = mask[:, :, :GEN_SEQ_LEN // BLOCK]
prefill_mask.seq_lengths = (prefill_q.shape[2], GEN_SEQ_LEN)
print(prefill_mask.to_string())
flex_attention(query=prefill_q, key=gen_kv, value=gen_kv, block_mask=prefill_mask)

(0, 0)
░░                              
██░░                            
████░░                          
██████░░                        
████████░░                      
██████████░░                    
████████████░░                  
██████████████░░                
████████████████░░              
██████████████████░░            
████████████████████░░          
██████████████████████░░        
████████████████████████░░      
██████████████████████████░░    
████████████████████████████░░  
██████████████████████████████░░



tensor([[[[ 4.1195e-01, -3.3926e-01, -6.8978e-01,  ..., -2.9023e-01,
            3.7244e-01,  5.4868e-02],
          [ 1.1261e-01, -2.4584e-01,  3.0970e-01,  ..., -1.1664e-01,
            3.5012e-01, -4.3909e-02],
          [ 1.5980e-01, -3.3563e-01, -6.4385e-02,  ..., -3.1995e-01,
            4.8191e-01, -2.4802e-02],
          ...,
          [-3.4188e-01, -9.1159e-02,  1.7803e-01,  ..., -1.1198e-01,
           -4.4493e-01, -2.6176e-02],
          [-1.1337e-01, -5.9948e-02,  3.1487e-01,  ...,  5.1692e-02,
            1.7198e-01, -3.6628e-01],
          [ 3.5240e-01, -3.1474e-01,  4.0829e-01,  ...,  1.6625e-01,
            3.7868e-01, -1.5443e-01]],

         [[-1.2704e-01,  1.2007e-01,  3.1669e-01,  ...,  2.7386e-01,
            8.3032e-03, -8.1694e-02],
          [-1.8151e-01,  6.7174e-01,  2.0796e-01,  ...,  9.2213e-02,
           -1.0149e-01, -3.4849e-01],
          [ 1.0899e-01, -2.1637e-02,  2.6686e-01,  ...,  3.8573e-01,
            6.6544e-02, -2.7641e-01],
          ...,
     

In [32]:
gen_mask = mask[:, :, PREFILL_LEN // BLOCK]
gen_mask.seq_lengths = (1, GEN_SEQ_LEN)
flex_attention(query=gen_q, key=gen_kv, value=gen_kv, block_mask=gen_mask)

tensor([[[[-1.4957e-01, -6.6806e-02,  5.3588e-01,  1.9895e-01, -3.8970e-01,
            4.1973e-01,  2.3263e-01, -1.6648e-01, -5.4415e-02,  3.3296e-01,
            1.0558e-01,  3.3214e-02, -8.6039e-01,  3.6053e-02,  8.0356e-01,
           -2.9449e-01]],

         [[ 2.2937e-01,  9.9052e-03,  2.7602e-01,  1.4465e-01, -7.8879e-03,
            6.5240e-02, -6.4973e-02,  4.1043e-01,  1.2808e-01, -1.0506e-01,
           -2.1053e-02, -8.9641e-02,  6.0654e-03, -2.7764e-02,  7.3124e-02,
            3.1082e-01]],

         [[-1.1311e-01,  1.9550e-01, -5.3326e-02, -1.2754e-01,  1.3930e-01,
           -2.6047e-04,  2.2431e-01, -6.6530e-03,  4.7019e-02, -3.6333e-01,
           -7.4459e-02, -7.1648e-02, -4.4526e-02,  7.6795e-02,  4.7296e-01,
           -6.1912e-01]],

         [[-8.9369e-01, -6.3822e-02,  2.2888e-01, -5.8779e-01, -5.5527e-02,
            3.8270e-01,  8.9829e-03, -1.8021e-01, -6.6043e-02,  1.5288e-01,
            1.9701e-01,  2.3214e-01, -1.4514e-01, -3.8047e-01,  2.0863e-01,
       